**Proyecto: Solución de SL DT**

IMPLEMENTACION DE ALGORITMOS

Integrantes: Ana Viñan, Kevin Bravo

In [ ]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 35.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import implicit
from scipy.sparse import csr_matrix
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
ruta_archivo = '/content/drive/My Drive/z1.csv'

Mounted at /content/drive


In [ ]:
df2 = pd.read_csv(ruta_archivo, sep=',', encoding="ISO-8859-1", low_memory=False)
df2.head(10)

,FECHA,COD_FACTURA,NOM_CLIENTE,SEXO,COD_SUCURSAL,COD_PRODUCTO,COD_PRODUCTO_TIA,CANTIDAD,VALOR_PVSI,COSTO,...,DESC_PRODUCTO,DESC_CLASE,DESC_SUBCLASE,DESC_SUCURSAL,MARCA,SECCION,DIA,MES,FRECUENCIA_COMPRAS,CLASIFICACION
0,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,251705000,25170500004,2.0,9.9800,4.1684,...,HUEVOS MEDIANOS 30 UNI,HUEVOS,HUEVOS GALLINA,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
1,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,149767000,149767000,1.0,0.8929,0.5100,...,TOALLAS DE COCINA HOME CLUB 180 UNI,DESCARTABLES DE COCINA,TOALLAS DE COCINA,MACHALA CENTRO,NaN,14,1,1,1,Baja frecuencia
2,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,259826000,259826000,1.0,1.9900,1.2500,...,CIRUELAS PASAS C/SEMILLA MAS AHORRO 200 G,FRUTAS SECAS,PASAS,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
3,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,141795000,141795000,1.0,1.4196,0.9905,...,FUNDA P/BASURA 76X91 CM ESTRELLA INDUSTRIAL 10...,DESCARTABLES DE COCINA,FUNDAS PARA BASURA,MACHALA CENTRO,NaN,14,1,1,1,Baja frecuencia
4,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,259824000,259824000,1.0,2.5900,1.5800,...,NUECES S/CASCARA MAS AHORRO 200 G,FRUTAS SECAS,NUECES,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
5,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,258141000,258141000,1.0,1.7768,1.2203,...,INFUSIONES NATURE'S HEART 20 SOBRES RED FRUITS,INFUSIONES,INFUSIONES,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
6,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,257482000,257482000,1.0,5.2589,3.0477,...,ACEITE DE OLIVA EXTRA VIRGEN LA ESPANOLA 250 ML,ACEITES,OLIVA,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
7,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,255519000,255519000,1.0,2.2500,1.4653,...,PAN MOLDE MODERNA 575 G INTEGRAL,PANIFICADOS,PAN DE MOLDE INTEGRAL,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
8,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,141648000,14164800003,1.0,1.7768,1.2500,...,PLANCHE FACIL 1 2 3 500 ML FLORAL,TRATAMIENTO PARA LA ROPA,PREPLANCHADO,MACHALA CENTRO,NaN,14,1,1,1,Baja frecuencia
9,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,331022000,331022000,1.0,3.9900,3.0151,...,LECHE LA LECHERA TETRABRIK 1 L X 3 SEMIDESCREMADA,LECHE CARTON UHT,LECHE CARTON SEMIDESCREMADA,MACHALA CENTRO,NaN,33,1,1,1,Baja frecuencia


In [ ]:
# Dimensiones del DataFrame
print("Dimensiones del DataFrame:", df2.shape)

Dimensiones del DataFrame: (698233, 24)


In [ ]:
df2_filtrado = df2[df2['SECCION'] == 25]

# Mostrar los primeros 5 resultados filtrados
df2_filtrado.head()

,FECHA,COD_FACTURA,NOM_CLIENTE,SEXO,COD_SUCURSAL,COD_PRODUCTO,COD_PRODUCTO_TIA,CANTIDAD,VALOR_PVSI,COSTO,...,DESC_PRODUCTO,DESC_CLASE,DESC_SUBCLASE,DESC_SUCURSAL,MARCA,SECCION,DIA,MES,FRECUENCIA_COMPRAS,CLASIFICACION
0,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,251705000,25170500004,2.0,9.9800,4.1684,...,HUEVOS MEDIANOS 30 UNI,HUEVOS,HUEVOS GALLINA,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
2,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,259826000,259826000,1.0,1.9900,1.2500,...,CIRUELAS PASAS C/SEMILLA MAS AHORRO 200 G,FRUTAS SECAS,PASAS,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
4,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,259824000,259824000,1.0,2.5900,1.5800,...,NUECES S/CASCARA MAS AHORRO 200 G,FRUTAS SECAS,NUECES,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
5,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,258141000,258141000,1.0,1.7768,1.2203,...,INFUSIONES NATURE'S HEART 20 SOBRES RED FRUITS,INFUSIONES,INFUSIONES,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
6,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,257482000,257482000,1.0,5.2589,3.0477,...,ACEITE DE OLIVA EXTRA VIRGEN LA ESPANOLA 250 ML,ACEITES,OLIVA,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia


In [ ]:
df2_filtrado.shape

(382170, 24)

In [ ]:
#FILTRAR EL TOP 100 DE PRODUCTOS

# Agrupar los productos por 'COD_PRODUCTO' y sumar las cantidades vendidas
top_100_vendidos = df2_filtrado.groupby('COD_PRODUCTO')['CANTIDAD'].sum().reset_index()

# Ordenar por la cantidad vendida en orden descendente
top_100_vendidos = top_100_vendidos.sort_values(by='CANTIDAD', ascending=False)

# Seleccionar los 100 productos más vendidos
top_100_vendidos = top_100_vendidos.head(100)

#Filtrar el df2_filtrado original para que solo contenga los 100 productos más vendidos
df2_filtrado_top_100 = df2_filtrado[df2_filtrado['COD_PRODUCTO'].isin(top_100_vendidos['COD_PRODUCTO'])]

# Mostrar los resultados filtrados
df2_filtrado_top_100.head()

,FECHA,COD_FACTURA,NOM_CLIENTE,SEXO,COD_SUCURSAL,COD_PRODUCTO,COD_PRODUCTO_TIA,CANTIDAD,VALOR_PVSI,COSTO,...,DESC_PRODUCTO,DESC_CLASE,DESC_SUBCLASE,DESC_SUCURSAL,MARCA,SECCION,DIA,MES,FRECUENCIA_COMPRAS,CLASIFICACION
7,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,255519000,255519000,1.0,2.2500,1.4653,...,PAN MOLDE MODERNA 575 G INTEGRAL,PANIFICADOS,PAN DE MOLDE INTEGRAL,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
12,2024-01-01 00:00:00+00:00,9001001863321,NERY FABIAN CABRERA TAPIA,1,109,255893000,255893000,1.0,1.6900,0.7500,...,LENTEJA DEL SUR 454 G,GRANOS,LENTEJA,MACHALA CENTRO,NaN,25,1,1,1,Baja frecuencia
14,2024-01-02 00:00:00+00:00,9001001863565,MARGARITA DEL CISNE SOLORZANO CAMPOVERDE,0,109,251625000,25162500002,1.0,0.8839,0.7421,...,DULCE DE LECHE KIOSKO 250 G,MANJAR DE LECHE,MANJAR DE LECHE,MACHALA CENTRO,NaN,25,5,1,1,Baja frecuencia
28,2024-01-02 00:00:00+00:00,9001001863565,MARGARITA DEL CISNE SOLORZANO CAMPOVERDE,0,109,257468000,25746800002,1.0,1.8100,1.6558,...,ACEITE CRIOLLO 900 ML,ACEITES,PALMA,MACHALA CENTRO,NaN,25,5,1,1,Baja frecuencia
31,2024-01-02 00:00:00+00:00,9001001863565,MARGARITA DEL CISNE SOLORZANO CAMPOVERDE,0,109,251699000,251699000,2.0,1.9800,0.7599,...,PAN MOLDE BLANCO FLOR DE ORO 450 G,PANIFICADOS,PAN DE MOLDE,MACHALA CENTRO,NaN,25,5,1,1,Baja frecuencia


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np

# DIVISION DE TRAIN Y TEST

# 1. Dividir el DataFrame de transacciones en entrenamiento y prueba
df_train, df_test = train_test_split(df2_filtrado_top_100, test_size=0.3, random_state=42)

# 2. Crear matrices de productos comprados juntos para cada conjunto
df_train_compras = df_train.groupby(['COD_FACTURA', 'COD_PRODUCTO'])['CANTIDAD'].sum().unstack().fillna(0)
df_test_compras = df_test.groupby(['COD_FACTURA', 'COD_PRODUCTO'])['CANTIDAD'].sum().unstack().fillna(0)

# Matriz de co-ocurrencia para el entrenamiento
df_train_matrix = df_train_compras.T.dot(df_train_compras)

In [ ]:
# IMPLEMENTACIÓN KNN

# 3. Entrenar el modelo KNN con los datos de entrenamiento
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(df_train_matrix)



NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# 4. Generar recomendaciones para los productos en el conjunto de prueba
knn_recommendations = {}

for product_id in df_test_compras.columns:
    if product_id in df_train_compras.columns:
        # Obtener las 5 recomendaciones más cercanas para el producto del conjunto de prueba
        knn_predicted = knn_model.kneighbors([df_train_matrix.loc[product_id].values], n_neighbors=6)[1].flatten()

        # Excluir el producto actual de las recomendaciones
        recommended_products = [df_train_compras.columns[i] for i in knn_predicted if df_train_compras.columns[i] != product_id]

        # Almacenar solo las 5 recomendaciones que no sean el producto actual
        knn_recommendations[product_id] = recommended_products[:5]
    else:
        # Si el producto no está en el conjunto de entrenamiento, proporcionar recomendaciones generales (productos más populares)
        most_common_products = df_train_compras.sum(axis=0).sort_values(ascending=False).index[:5]
        knn_recommendations[product_id] = most_common_products

In [ ]:
# 5. Mostrar las recomendaciones generadas por KNN
for product_id, recommended_products in knn_recommendations.items():
    print(f"Recomendaciones KNN para el producto {product_id}: {recommended_products}")


Recomendaciones KNN para el producto 250137000: [250209000, 250218000, 252223000, 255302000, 258742000]
Recomendaciones KNN para el producto 250206000: [257407000, 253277000, 259782000, 257434000, 257708000]
Recomendaciones KNN para el producto 250209000: [250137000, 252223000, 250218000, 255302000, 258743000]
Recomendaciones KNN para el producto 250218000: [252223000, 250137000, 250209000, 253278000, 257676000]
Recomendaciones KNN para el producto 250421000: [255843000, 257354000, 251305000, 257464000, 257708000]
Recomendaciones KNN para el producto 250471000: [251308000, 259852000, 257464000, 257708000, 259853000]
Recomendaciones KNN para el producto 250595000: [257476000, 257464000, 257434000, 251305000, 257708000]
Recomendaciones KNN para el producto 250643000: [257755000, 257464000, 251628000, 255564000, 253277000]
Recomendaciones KNN para el producto 251214000: [257464000, 251305000, 255843000, 257354000, 253278000]
Recomendaciones KNN para el producto 251221000: [257464000, 2513

In [ ]:
#MÉTRICAS
# 1. Definir funciones para calcular Precision@K, Recall@K y F1-score@K
def precision_at_k(recommendations, ground_truth, k=5):
    precision_scores = []

    # Para cada producto, calcular Precision@K
    for product_id, recommended_products in recommendations.items():
        if product_id in ground_truth:
            # Obtener los productos relevantes (comprados juntos)
            relevant_products = ground_truth[product_id]

            # Productos recomendados dentro del top k
            recommended_top_k = set(recommended_products[:k])

            # Intersección de productos recomendados y productos relevantes
            num_relevant_in_top_k = len(recommended_top_k.intersection(relevant_products))

            # Calcular la precisión como la proporción de productos relevantes en las recomendaciones
            precision = num_relevant_in_top_k / min(k, len(recommended_top_k)) if len(recommended_top_k) > 0 else 0
            precision_scores.append(precision)

    # Promedio de precisión
    return np.mean(precision_scores) if len(precision_scores) > 0 else 0


def recall_at_k(recommendations, ground_truth, k=9):
    recall_scores = []

    # Para cada producto, calcular Recall@K
    for product_id, recommended_products in recommendations.items():
        if product_id in ground_truth:
            # Obtener los productos relevantes (comprados juntos)
            relevant_products = ground_truth[product_id]

            # Productos recomendados dentro del top k
            recommended_top_k = set(recommended_products[:k])

            # Intersección de productos recomendados y productos relevantes
            num_relevant_in_top_k = len(recommended_top_k.intersection(relevant_products))

            # Calcular el recall como la proporción de productos relevantes encontrados en las recomendaciones
            recall = num_relevant_in_top_k / len(relevant_products) if len(relevant_products) > 0 else 0
            recall_scores.append(recall)

    # Promedio de recall
    return np.mean(recall_scores) if len(recall_scores) > 0 else 0


def f1_score_at_k(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0



def calculate_rmse(ground_truth, recommendations, k=5):
    y_true = []
    y_pred = []

    for product_id, relevant_products in ground_truth.items():
        if product_id in recommendations:
            recommended_products = recommendations[product_id][:k]

            # Para cada producto recomendado, si está en los productos relevantes: 1, si no: 0
            y_true.extend([1 if p in relevant_products else 0 for p in recommended_products])

            # Usar una puntuación ficticia de 1 para las recomendaciones hechas por el modelo
            y_pred.extend([1] * len(recommended_products))

    # Calcular RMSE
    return np.sqrt(mean_squared_error(y_true, y_pred))


In [ ]:
# 2. Definir el Ground Truth (productos que efectivamente se compraron juntos en el conjunto de prueba)
ground_truth = {}

# Para cada producto en el conjunto de prueba, almacenar los productos comprados juntos como "ground truth"
for invoice_id, products in df_test.groupby('COD_FACTURA')['COD_PRODUCTO']:
    for product in products:
        if product not in ground_truth:
            ground_truth[product] = set(products) - {product}
        else:
            ground_truth[product].update(set(products) - {product})


In [ ]:
# 3. Calcular las métricas Precision@K, Recall@K y F1-score@K
precision = precision_at_k(knn_recommendations, ground_truth, k=5)
recall = recall_at_k(knn_recommendations, ground_truth, k=5)
f1 = f1_score_at_k(precision, recall)
rmse = calculate_rmse(ground_truth,knn_recommendations, k=5)


# 4. Mostrar las métricas
print(f"Precision@5 (KNN): {precision:.4f}")
print(f"Recall@5 (KNN): {recall:.4f}")
print(f"F1-Score@5 (KNN): {f1:.4f}")
print(f"RMSE (KNN): {rmse:.4f}")

Precision@5 (KNN): 0.9920
Recall@5 (KNN): 0.0525
F1-Score@5 (KNN): 0.0998
RMSE (KNN): 0.0894


In [ ]:
# IMPLEMENTACION ALS
import implicit
import numpy as np
from scipy.sparse import csr_matrix

# Convertir los datos de entrenamiento a una matriz dispersa CSR (df_train_compras se usa en lugar de df_train_filtered)
df_train_sparse = csr_matrix(df_train_compras.values)

# Crear el modelo ALS
als_model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=30)

# Entrenar el modelo ALS
als_model.fit(df_train_sparse)

# Crear diccionario para almacenar las recomendaciones
als_recommendations = {}

# Para cada producto en el conjunto de prueba, generar recomendaciones
for product_id in df_test_compras.columns:  # Usamos el conjunto de test adecuado
    if product_id in df_train_compras.columns:  # Verificamos si el producto está en el conjunto de entrenamiento (df_train_compras)
        # Obtener el índice del producto en df_train_compras
        product_idx = df_train_compras.columns.get_loc(product_id)

        # Obtener las recomendaciones para el producto (omitiendo el propio producto)
        recommended_products = als_model.recommend(product_idx, df_train_sparse.T, N=6, filter_already_liked_items=False)

        # Filtrar el producto actual y almacenar las recomendaciones
        recommended_products_list = []
        for i, score in zip(recommended_products[0], recommended_products[1]):
            if df_train_compras.columns[i] != product_id:
                recommended_products_list.append(df_train_compras.columns[i])

        # Guardar las recomendaciones (máximo 5)
        als_recommendations[product_id] = recommended_products_list[:5]
    else:
        # Si el producto no está en el conjunto de entrenamiento, proporcionar recomendaciones generales (productos más populares)
        most_common_products = df_train_compras.sum(axis=0).sort_values(ascending=False).index[:5]
        als_recommendations[product_id] = most_common_products

    # Imprimir las recomendaciones para verificar
    print(f"Recomendaciones ALS para el producto {product_id}: {als_recommendations[product_id]}")

# Remueve el break para que procese todos los productos



/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/30 [00:00<?, ?it/s]

Recomendaciones ALS para el producto 250137000: [255893000, 258742000, 258743000, 256139000, 255560000]
Recomendaciones ALS para el producto 250206000: [257468000, 251625000, 258756000, 257388000, 255560000]
Recomendaciones ALS para el producto 250209000: [257426000, 253247000, 250595000, 258047000, 253166000]
Recomendaciones ALS para el producto 250218000: [255560000, 257680000, 257760000, 257676000, 257473000]
Recomendaciones ALS para el producto 250421000: [257468000, 257260000, 258756000, 251313000, 259852000]
Recomendaciones ALS para el producto 250471000: [259789000, 251625000, 257499000, 255560000, 250643000]
Recomendaciones ALS para el producto 250595000: [257354000, 257470000, 255544000, 251214000, 257464000]
Recomendaciones ALS para el producto 250643000: [259782000, 259783000, 259755000, 257755000, 258044000]
Recomendaciones ALS para el producto 251214000: [259628000, 251221000, 255560000, 250471000, 255564000]
Recomendaciones ALS para el producto 251221000: [258038000, 2555

In [ ]:

# 3. Calcular las métricas Precision@K, Recall@K y F1-score@K para ALS
precision_als = precision_at_k(als_recommendations, ground_truth, k=5)
recall_als = recall_at_k(als_recommendations, ground_truth, k=5)
f1_als = f1_score_at_k(precision_als, recall_als)
rmse_als = calculate_rmse(ground_truth,als_recommendations, k=5)

# 4. Mostrar las métricas
print(f"Precision@5 (ALS): {precision_als:.4f}")
print(f"Recall@5 (ALS): {recall_als:.4f}")
print(f"F1-Score@5 (ALS): {f1_als:.4f}")
print(f"RMSE (ALS): {rmse_als:.4f}")


Precision@5 (ALS): 0.9480
Recall@5 (ALS): 0.0500
F1-Score@5 (ALS): 0.0951
RMSE (ALS): 0.2280


In [ ]:
#NCF

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, Dense, Concatenate
from sklearn.model_selection import train_test_split

# Añadir una columna de interacción para entrenamiento (1 indica interacción)
df2_filtrado_top_100['interaction'] = 1  # Añadir una columna de interacción para el entrenamiento

# Crear un mapeo de productos a índices consecutivos
unique_products = df2_filtrado_top_100['COD_PRODUCTO'].unique()
product_to_index = {product: idx for idx, product in enumerate(unique_products)}

# Aplicar el mapeo a los datos
df2_filtrado_top_100['product_index'] = df2_filtrado_top_100['COD_PRODUCTO'].map(product_to_index)

# Definir el tamaño del vocabulario (número de productos únicos)
vocab_size = len(product_to_index)

# Dividir los datos en entrenamiento y prueba
train_data, test_data = train_test_split(df2_filtrado_top_100, test_size=0.2, random_state=42)

# Definir el modelo de recomendación de producto a producto
input_product = Input(shape=(1,))
embedding_product = Embedding(input_dim=vocab_size, output_dim=64)(input_product)
flatten_product = Flatten()(embedding_product)

# Simetría en el segundo producto (para recomendar productos entre sí)
input_product_2 = Input(shape=(1,))
embedding_product_2 = Embedding(input_dim=vocab_size, output_dim=64)(input_product_2)
flatten_product_2 = Flatten()(embedding_product_2)

# Concatenación de las representaciones embebidas de ambos productos
concatenated = Concatenate()([flatten_product, flatten_product_2])

# Agregar capas densas
dense_1 = Dense(128, activation='relu')(concatenated)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(vocab_size, activation='softmax')(dense_2)  # Salida de softmax para obtener probabilidades

# Compilar el modelo
model = Model(inputs=[input_product, input_product_2], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    [train_data['product_index'], train_data['product_index']],
    train_data['product_index'],  # Usar product_index como etiquetas
    validation_data=([test_data['product_index'], test_data['product_index']], test_data['product_index']),
    epochs=10,
    batch_size=64
)

# Evaluar el modelo
evaluation = model.evaluate([test_data['product_index'], test_data['product_index']], test_data['product_index'])
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")

# Hacer predicciones
predicciones = model.predict([test_data['product_index'], test_data['product_index']])

# Crear un DataFrame de predicciones a nivel de COD_PRODUCTO
predicciones_df = pd.DataFrame(predicciones, columns=[f'prob_{product}' for product in unique_products])

# Obtener las recomendaciones para cada producto en el conjunto de prueba
def obtener_recomendaciones(predicciones, product_index, top_n=5):
    """Obtener los mejores productos recomendados para un producto dado."""
    recomendaciones = np.argsort(predicciones[product_index])[-top_n:][::-1]  # Obtener los índices de los mejores productos
    productos_recomendados = [unique_products[i] for i in recomendaciones]  # Mapear a COD_PRODUCTO
    return productos_recomendados

# Crear un DataFrame para almacenar las recomendaciones
recomendaciones_todas = {}

for idx in range(len(test_data)):
    producto_actual = test_data['COD_PRODUCTO'].iloc[idx]
    product_index = test_data['product_index'].iloc[idx]
    recomendaciones_todas[producto_actual] = obtener_recomendaciones(predicciones, product_index)

# Imprimir las recomendaciones para verificar
for ite, reco in recomendaciones_todas.items():
    print(ite, reco)

<ipython-input-19-4fa012ad83a4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_filtrado_top_100['interaction'] = 1  # Añadir una columna de interacción para el entrenamiento
<ipython-input-19-4fa012ad83a4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_filtrado_top_100['product_index'] = df2_filtrado_top_100['COD_PRODUCTO'].map(product_to_index)


Epoch 1/10
1992/1992 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8993 - loss: 0.7513 - val_accuracy: 1.0000 - val_loss: 1.1937e-04
Epoch 2/10
1992/1992 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 1.0000 - loss: 7.8879e-05 - val_accuracy: 1.0000 - val_loss: 2.1680e-05
Epoch 3/10
1992/1992 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 1.0000 - loss: 1.5967e-05 - val_accuracy: 1.0000 - val_loss: 5.9030e-06
Epoch 4/10
1992/1992 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 1.0000 - loss: 4.4954e-06 - val_accuracy: 1.0000 - val_loss: 1.8181e-06
Epoch 5/10
1992/1992 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 1.0000 - loss: 1.3975e-06 - val_accuracy: 1.0000 - val_loss: 5.6537e-07
Epoch 6/10
1992/1992 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 1.0000 - loss: 4.3319e-07 - val_accuracy: 1.0000 - val_loss: 1.4997e-07
Epoch 7/10
1992/1992 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 1.0000 - loss: 1.1285e-07 - val_accuracy: 1.0000 - val_loss: 4.8329e-08
Epoch 8/10
1992/1992 ━━━━━━━━━━━━━━━

In [ ]:
# 3. Calcular las métricas Precision@K, Recall@K y F1-score@K para ALS
precision_ncf = precision_at_k(recomendaciones_todas, ground_truth, k=5)
recall_ncf = recall_at_k(recomendaciones_todas, ground_truth, k=5)
f1_ncf = f1_score_at_k(precision_ncf, recall_ncf)
rmse_ncf = calculate_rmse(ground_truth, recomendaciones_todas, k=5)

# 4. Mostrar las métricas
print(f"Precision@5 (NCF): {precision_ncf:.4f}")
print(f"Recall@5 (NCF): {recall_ncf:.4f}")
print(f"F1-Score@5 (NCF): {f1_ncf:.4f}")
print(f"RMSE (NCF): {rmse_ncf:.4f}")


Precision@5 (NCF): 0.9580
Recall@5 (NCF): 0.0506
F1-Score@5 (NCF): 0.0962
RMSE (NCF): 0.2049
